In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Segmentation

Image patch classifier
<img src="images/od/semantic_segmentation_sliding_window.png" height="1000" width="1000">

Computationally expensive

Image patch classifier
<img src="images/od/semantic_segmentation_sliding_window.png" height="1000" width="1000">

Computationally expensive and hard to classify without additional information of image context

Fully convolutional model byt at the end layers information is lost
<img src="images/od/fully_conv.jpeg" height="1000" width="1000">

Fully convolutional model size preserving
<img src="images/od/fully_convolutional_2.png" height="1000" width="1000">

Without down-sampling preserve original image scale would be really expensive and that's why encoder-decoder model is preferred

Down-sampling plus up-sampling (Encoder decoder architecture)
<img src="images/od/encoder_decoder_1.png" height="1000" width="1000">

Encoder-decoder application:
<img src="images/od/encoder_decoder_2.png" height="1000" width="1000">

- Detects smaller objects on first layers of encoder and last layers on decoder
- Detects bigger objects on last layers of encoder and fist layers on decoder
- Decodes medium sized objects on medium layers
- Example auto-encoders

<img src="images/od/autoencoder.png" height="1000" width="1000">

## Up-sampling

Unpooling example (non-learnable):
<img src="images/od/unpooling_1.png" height="1000" width="1000">

Recall convolution with stride:
<img src="images/od/conv_stride_1.png" height="1000" width="1000">

Transpose convolution:
<img src="images/od/transpose_1.png" height="1000" width="1000">

Transpose convolution for $1$ dimension:
<img src="images/od/transpose_2.png" height="1000" width="1000">

After each layer naturally there are activations, maybe normalization and other layers

If we write layer as matrix operation:
<img src="images/od/transpose_matrix_1.png" height="1000" width="1000">

Transpose convolution because of it's nature of repeating the same value from receptive field, causes the so called "checkerboard" artifacts:
<img src="images/od/checkerboard_1.png" height="600" width="600">

Other up-sampling methods:
Up-sample with chosen (bilinear) interpolation for instance twice or three times and then run convolution:
<br>
$$
I_r = \frac{I - F_d + 2 P}{S_d} + 1 = \frac{I}{S_d} - \frac{F_d - 2 P}{S_d} + 1
$$
<br>
so we get:
$$
I_r = \frac{n \cdot I - F_d + 2 P}{S_d} + 1 = n \cdot \frac{I}{S_d} - \frac{I}{F_d - 2 P}{S_d} + 1
$$

There are bunch of other learnable up-sampling methods, for instance pixel shuffle up-sampling instead of other interpolation before convolution layer:
<img src="images/od/pixel_shuffle_1.jpg" height="1000" width="1000">

The main results are:
- Up-sample might be learnable layer
- It's pretty similar to convolution layer with activations 
- Has the same depth for filter as input and outputs no depth
- Might generate as match up-sampled feature maps as we configure

During the down-sample convolutional layers, information is extracted from the input image independent from the tasks sometimes.
<br>
Often pre-trained (on ImageNet classification task) encoder (often called backbone) used

## Loss

For segmentation we can use $L_2$ or $L_1$ loss:
$$
L_2 = \sum_{i = 1}^{h}\sum_{j=1}^{w}||y_{ij} - \hat{y}_{ij}||^2
$$
<br>
$$
L_1 = \sum_{i = 1}^{h}\sum_{j=1}^{w}||y_{ij} - \hat{y}_{ij}||
$$

We can produce as many feature maps on last layer as many objects we have
<br>
or different number can denote different objects for labels:
$$\begin{align} Y &= \begin{pmatrix}
           0, 1, \dots, 0 \\
           1, 1, \dots, 0 \\
           \vdots \\
           0, 1, \dots, 1 \\
         \end{pmatrix}
  \end{align}$$
  <br>
  or:
  $$\begin{align} Y &= \begin{pmatrix}
           1, 2, \dots, 4 \\
           0, 4, \dots, 2 \\
           \vdots \\
           2, 1, \dots, 0 \\
         \end{pmatrix}
  \end{align}$$

## Why not mix encoder-decoder models in segmentation-recognition models

Mix encoder-decoder model layers
<img src="images/od/mixing_layers_1.png" height="800" width="800">

## UNet combine encoder and decoder

Lets combing down-sampling and up-sampling

UNet
<img src="images/od/unet.png" height="800" width="800">

Lets combing down-sampling and up-sampling

UNet
<img src="images/od/unet_2.png" height="800" width="800">

## Single shot detection

Multi model has high performance but at the same time:
- they are hard to train
- hard to select correct parameters / hyper-parameters
- computationally expensive during the training
- computationally expensive during the inference

For the real time or near-real time approaches there exists different single shot detectors
Which runs bounding box regression and the classification at the same time

<img src="images/od/localization_classification.png" height="1000" width="1000">

In some aspects, this is the main approach, we can not have a undefined output of the feed-forward neural networks (not RNN)
<br>
So we need somehow localize single object in the image
<br>

First approach which we saw in the previous presentation was the family of the RCNN models
<br>
Model first detected blobs with different algorithms (Region proposals, Region proposals on the feature map, decoder encoder-model) and then uses classification + localization algorithm

On the other hand we can split the image by the smaller parts for instance $10 \times 10$ or $6 \times 6$ or even $19 \times 19$ parts, grid-cells

<img src="images/od/yolo_1.png" height="600" width="600">

For the object detection we have:
- Object classes: $(c_1, c_2, \dots, c_n)$
- Bounding boxes: $((x, y, h, w )_1, (x, y, h, w )_2 \dots (x, y, h, w )_m)$

And for each grid cell (which is small enough) lets take center and classify is there object or not:
- $p$ is the probability is there object or not
- Also class label $𝑐_1,𝑐_2,…,𝑐_𝑛$
- Bounding boxes $x, y, h, w $ for this instant class

So we have:
$$
\begin{align}
    g_i &= \begin{bmatrix}
           p \\
           x \\
           y \\
           h \\
           w \\
           c_1 \\
           c_2 \\
           \vdots \\
           c_n
         \end{bmatrix}
  \end{align}
$$
<br>
For each grid-cell of image

Now let consider it as a tensor with $6 \times 6$, $14 \times 14$, $19 \times 19$, $g \times g$ or any other, with $g_i$ grid cell vector for each
<br>
$g \times g \times (5 + c)$
- So use this as a target for the prediction and run model per grid-cell
- Or we can use compose our model with the single model

So lets create neural network which has a convolutional layers with the our $g \times g$ model
<img src="images/od/yolo_2.gif" height="600" width="600">

## YOLO-v3

Previous YOLO models does not have up-sampling architectures which means that it was fast but it was not accurate enough
<br>
In YOLOv3 was added DarkNet53 backbone model

YOLOv3 model architecture with upsampling layers:
<img src="images/od/yolo_v3_arch.png" height="1000" width="1000">

## Anchor boxes

Recall that YOLOv3 was detecting objects for each cell and was assigning 
$$  \begin{align}
    y &= \begin{pmatrix}
           p \\
           x \\
           y \\
           h \\
           w \\
           c_{1} \\
           c_{2} \\
           \vdots \\
           c_{n}
         \end{pmatrix}
  \end{align}
$$
<br>
Let's imagine the scenario where two objects have the same center. Then our detection should decide by probability. But what if both of them are in the image?

Per cell we had a sigle vector:
<img src="images/od/yolo_pred_1.jpeg" height="1000" width="1000">

For instance person and car have the same center (no the best place for center):
<img src="images/od/anchor_1.jpeg" height="600" width="600">

Or the same here:
<img src="images/od/anchor_2.jpg" height="600" width="600">

If we observe the shape of the objects, the car of them has bigger width and smaller height while the person has smaller width and bigger height
<br>
Generally different type if objects has different shape and size and they might be assigned to the detection algorithm
<img src="images/od/anchor_3.png" height="600" width="600">

Let's use this information and add two predefined shapes to our algorithm, instead of one vector per cell, let's concatenate two of them, one pre-shape which we'll call anchor box:
$$  \begin{align}
    y_1 = \begin{pmatrix}
           p \\
           x \\
           y \\
           h \\
           w \\
           c_{1} \\
           c_{2} \\
           \vdots \\
           c_{n}
         \end{pmatrix}
\quad
    y_2 = \begin{pmatrix}
           p \\
           x \\
           y \\
           h \\
           w \\
           c_{1} \\
           c_{2} \\
           \vdots \\
           c_{n}
         \end{pmatrix}
  \end{align}
$$

Concatenate them and we will get:
$$ \begin{align}
    y = \begin{pmatrix}
           p \\
           x \\
           y \\
           h \\
           w \\
           c_{1} \\
           c_{2} \\
           \vdots \\
           c_{n} \\
           p \\
           x \\
           y \\
           h \\
           w \\
           c_{1} \\
           c_{2} \\
           \vdots \\
           c_{n}
         \end{pmatrix}
  \end{align}
$$

Or maybe we'll group probabilities, together or probabilities and bounding boxes, etc or put in other order, it does not matter:
$$ \begin{align}
    y = \begin{pmatrix}
           x \\
           y \\
           h \\
           w \\
           c_{1} \\
           c_{2} \\
           \vdots \\
           c_{n} \\
           x \\
           y \\
           h \\
           w \\
           c_{1} \\
           c_{2} \\
           \vdots \\
           c_{n} \\
           p_1 \\
           p_2
         \end{pmatrix}
  \end{align}
$$

We will have $19 \times 19 \times 2 \times {(1 + 2 + 4 + n)}$ dimensional output
<br>
Or as tensor $19 \times 19 \times {2 \cdot (1 + 2 + 4 + n)} = 19 \times 19 \times {14 + 2 n}$
<img src="images/od/yolo_2.gif" height="600" width="600">
This is the answer of the question from the previous presentation

In practice there are more than two anchor boxes for detection models:
<img src="images/od/pred_anchor_1.png" height="600" width="600">

Actually anchor boxes are predefined and used as default during the training.
During the training annotated object and it's bounding box is assigned to the concrete anchor box by the highest IoU value

People might generate anchor boxes by hand or use KNN or other advanced techniques for this.

The main flaw of anchor boxes is if there are more object with the same center that anchor boxes.
<br>
This is the case which anchor boxes approach can not deal with
<br>
But usually there are many enough anchor boxes and small enough cells in YOLO case to make this event almost impossible
<br>
For instance $19 \times 19$ cells are small enough with $9$ anchor boxes to avoid $9$ different object with the same center

Now the anchor boxes are used in many detection algorithms. Just increase output value by the fixed number

## Non-max suppression

Generally detector models predict multiply bounding boxes per object (per image) for instance YOLO might consider several several cells as center of the instant object:
<img src="images/od/non_max_1.jpg" height="600" width="600">

We can filter them by confidence and IoU thresholds, but still there will be many of them:
<img src="images/od/non_max_2.png" height="1000" width="1000">

Also, there might be many same category of object on a single image. How can we reduce number of boxes:
<br>
For a single object category:
- First remove all boxes with lower confidence that threshold
- Second we choose box with highest confidence
- Then calculate IoU with other boxes and if IoU is higher than $0.5$ for instance, remove this box
- Then do the same with the rest of the boxes

Remove low confidence, sort and then run non max suppression:
<img src="images/od/non_max_det_1.png" height="1000" width="1000">

Leave only low IoU box scores others put $0$ values and filter
<img src="images/od/non_max_det_2.png" height="1000" width="1000">

If we have a single object of the category:
<img src="images/od/non_max_3.png" height="1000" width="1000">

But why do we leave lower IoU images?
<br>
Because they might be the boxes for the different object of the same category:
<img src="images/od/non_max_4.png" height="1000" width="1000">

After we remove boxes with higher IoU we'll distinguish different objects of the same category:
<img src="images/od/non_max_5.png" height="1000" width="1000">

Here's the other example of different of the same category:
<img src="images/od/non_max_7.jpeg" height="1000" width="1000">

Multi-centers:
<img src="images/od/non_max_8.jpg" height="800" width="800">

## Questions
<img src="images/od/questions_1.jpg" height="800" width="800">

## VGG-16

Before we start describing SSD model let's have a quick look on VGG-16 architecture which is used as a backbone or feature extractor for SSD model

Model VGG (visual geometry group) was first presented at 2013 ImageNet competition and got high results (not winner ZENet was winner) but it was fast and had a simple architecture.
VGG-16 is still used in
- Backbone for many models
- Perceptual loss for style transfer and auto-encoder (super-resolution, variational auto-encoder) models
- From my experience, VGG-16 has a place, color and size invariant feature extraction property because of pooling layers and convolutional stack.

VGG-16 architecture:
<img src="images/od/vgg_16_1.png" height="800" width="800">

One of the best features extractor for background jobs:
- High dimension
- Spark job for image clustering

## SSD (Single shot multi box detection)

SSD is one of the single shot detection family members.
<br>
For object detection, as we remember from localization models, we need fixed number of bounding boxes.
<br>
SSD does this by assigning $n$ bounding boxes per pixel on feature map
<br>
Why on features map, because they have smaller number of pixels, smaller special (height and with) size
<br>
The number of bounding boxes might be different for the layer


General architecture with non max suppression:
<img src="images/od/ssd_2.png" height="1000" width="1000">

Detector and classifier block:
<img src="images/od/ssd_3.png" height="1000" width="1000">

We have a feature map with low spatial size for block, generate three bounding boxes per pixel:
<img src="images/od/ssd_4.png" height="1000" width="1000">

With corner correction:
<img src="images/od/ssd_7.png" height="1000" width="1000">

Train with appropriated loss:
<img src="images/od/ssd_10.png" height="1000" width="1000">

Different amount of bounding boxes per layer and correction at the end:
<img src="images/od/ssd_12.png" height="1000" width="1000">

## ResNet and notion of the skip-connections

ResNet by the Microsoft was winner of the 2015 ImageNet competition and the first model which outperformed the human

The main idea behind the ResNet architecture is so called skip-connections or residual blocks:


Residual block:
<img src="images/od/resnet_1.png" height="800" width="800">

With different interpretations:
<img src="images/od/resnet_2.png" height="800" width="800">

With the skip connections models models depth limit was increased with performance gain

ResNet vs other models
<img src="images/od/resnet_3.png" height="800" width="800">

Almost all modern architectures for classification (which are used as backbones), or encoders use some kind of modification of residual connections:
- Ineption-ResNet
- ResNeXt
- MobileNet
- EfficientNet

Residual connections in other architectures:
<img src="images/od/resnet_4.jpg" height="800" width="800">

## Feature pyramid models

Lets combing down-sampling and up-sampling

UNet
<img src="images/od/unet.png" height="800" width="800">

Let's make it slightly faster

<img src="images/od/fpn_1.jpeg" height="800" width="800">

<img src="images/od/fpn_2.jpeg" height="800" width="800">

## RetinaNet

Feature pyramids with ResNet50 as backbone:

<img src="images/od/retina_1.jpg" height="1000" width="1000">

Feature pyramid extracts and preserves semantically and spatially, after some feature pyramid block (two) predictors are applied, bounding box regression with $H \times W$ spacial size and $4A$ channels and classification networks with $H \times W$ spacial size and $KA$ channels, class per anchor box:

<img src="images/od/retina_2.jpg" height="1000" width="1000">

And classification:
<img src="images/od/retina_3.png" height="1000" width="1000">

Regression loss:
$$
\begin{align}
T^i_x &= (G^i_x - A^i_x) / A^i_w  \\
T^i_y &= (G^i_y - A^i_y) / A^i_h \\
T^i_w &= log(G^i_w / A^i_w) \\
T^i_h &= log(G^i_h / A^i_h)
\end{align}
$$

And loss is applied as:
$$
L_{loc} = \sum_{j \in \{x, y, w, h\}}smooth_{L1}(P^i_j - T^i_j)
$$
<br>
where:
$$
\begin{equation}
smooth_{L1}(x) = 
\begin{cases}
0.5x^2 &|x| < 1 \\
|x| - 0.5 &|x| \geq 1
\end{cases}
\end{equation}
$$
<br>
and
$$
P^i = (P^i_x, P^i_y, P^i_w, P^i_h)
$$

Focal loss:
$$
L_{cls} = -\sum_{i=1}^{K}(y_ilog(p_i)(1-p_i)^\gamma \alpha_i + (1 - y_i)log(1 - p_i)p_i^\gamma (1 - \alpha_i))
$$

Here we have two dependent hyper-parameters:
- Weighting parameter $\alpha$ which is responsible for class imbalance $\alpha_i \in [0, 1]$
- Focusing parameter $\gamma$ for background foreground distinguish $\gamma \in (0, +\infty)$
- During the weight initialization, biases on the last layer are initialized bigger with some rules

This parameters are balancing for meaning hard negatives.
<br>
Because of big amount of background images, data might be classified as background easily

Here's the example of training with and without focal loss:
<img src="images/od/focal_1.png" height="800" width="800">

At the end we'll add that each detector outputs $1$K boxes and then reduced by the non max suppression algorithm 

## Multi-shot object detection

- Problems with detecting small objects
- Different size of sliding windows
- Run classifier network per patch
- Computationally expensive

#### Region proposals

With classical CV (greedy) algorithm calculated regions of images - Selective Search
<img src="images/od/selective_search.jpg" height="800" width="800">

#### R-CNN

We calculate 2000 region proposals and run them through the pre-trained cnn classifier (VGG-16 or ResNet-50)
<img src="images/od/r_cnn_1.png" height="800" width="800">

We train SVM classifiers per extracted features and also BBox regression to improve boxes:
<img src="images/od/r_cnn_2.png" height="800" width="800">

- Improves accuracy
- Computationally expensive for training
- Computationally expensive for testing / inference
- Slow

#### Fast-RCNN

Instead of calculation of region proposals on input image, we first run image through CNN model and calculate region proposals on smaller feature map

Region proposals have high recall and many of them are classified as background

Region proposals $N \times 5$ image index and coordinates
<img src="images/od/region_proposal_cat.png" height="800" width="800">

Then we use ROI pooling to adjust features and flatten pooled image parts for classification
ROI poolint takes a section of the input feature map that corresponds to it and scales it to some pre-defined size
<img src="images/od/roi_pooling_1.gif" height="800" width="800">

<img src="images/od/fast_r_cnn.png" height="800" width="800">

- Less computational resources
- Faster training time
- Faster testing / inference time
- The same performance

#### Faster-RCNN

Instead of using selective search, let's use separate convolutional neural network which calculates region proposals

<img src="images/od/faster_r_cnn_1.png" height="800" width="800">

<img src="images/od/faster_r_cnn_2.png" height="800" width="800">

- Less computationally expensive
- Region proposals calculation is learnable
- Faster for training
- Faster for test / inference

## Questions
<img src="images/od/questions_2.jpg" height="800" width="800">

## Thank you